<a href="https://colab.research.google.com/github/xgpo-2226/EFFDL_Semaine1/blob/main/Labsheets/Core/L3_MotionControl_SEMTM0042_43.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 3: Motion Control


**Aims:**

In this labsheet we will learn about kinematics, odometry, and how to use both for motion control.  In particular we will need to understand:

- The **local** and **global** coordinate frame.
- The roles of parameters in the **kinematic model**.
- The necessity of **calibrating** the kinematic model.
- The relationship between **robot behaviours**, the kinematic model, and sources of **odometric error**.

**Objectives:**

At this stage, we need the robot to:
 - Track it's position over time (**odometry**).
 - To rotate to face specific directions.
 - To travel until a target is reached.
 - To join rotation and travel behaviours together so that from any position on the coursework map, the robot can efficiently move to another position.


**Context:**

You have probably noticed that the above objectives are very similar to the previous labsheet.  In the previous labsheet, we wrote some **open-loop** code to perform simple rotation and translation based on a time duration in milliseconds.  By the end of the previous labsheet, you will have also written **closed-loop** PID control for wheel speed.

In this labsheet, we will write rotation and translation behaviours that are **closed-loop**.  For example, this means we want the robot to rotate to a specific angle of orientation.  As closed-loop control, if the robot was to rotate too far, we would expect it to automatically attempt to rotate back.

To do this, we need an information source to act as our measurement, and a target to act as the demand.  For both, we can utilise the **rotary encoder** counts to estimate the change of the position of the robot over time, referred to as **odometry**. Doing so means that the robot can utilise a cartesian coordinate system (i.e., $x$, $y$ and $\theta$) to navigate and control it's motion.  If we are going to use **odometry**, then it is important that the underlying kinematic model is accurate.


<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  When you approach this labsheet, you should adopt a plan of breaking the work up into small steps.  For example, a good way to think about the challenges of this labsheet in small steps (goals) is:


a.  Getting the robot to rotate a specific angle

b.  Getting the robot to travel a specific distance

c.  Getting the robot to first rotate, and then travel a distance

d.  Calculating an angle to a point (x,y), and performing (a)

e.  Calculating the distance to a point (x,y), and performing (b)

d.  Linking (d), (e), and (c) to perform a "go to x,y" behaviour.

e.  Linking (d) recursively, to move between two points repeatedly.

f.  Improving (c) so that it can correct for errors in rotation whilst it is travelling a distance.


<hr><br><br><br><br>


<hr><br><br><br><br>

# Kinematics




As a starting point, we want to become familiar with the coordinate system.  Once we have an understanding of this, we can progress to **calibrating**, **rotating** and **translation**.

A simple kinematic model has been provided for you in the <a href="https://github.com/paulodowd/SEMTM0042_43/tree/main/3Pi_CodeStub">code stub</a> available on GitHub.  The kinematic model is contained in the header file `Kinematics.h`.  The kinematics is utilising the change in encoder counts over time to estimate the translation and rotation of the robot. Therefore, `Kinematics.h` **depends on** `Encoders.h`.  The smallest sketch you could produce to use the kinematics is given as an example below. Start by reviewing the below code, read the comments:

```c
#include "Encoders.h"   // provide automatic counting of encoders
#include "Kinematics.h" // maths for odometry/dead-reckoning

// Create a global instance of the kinematics class
Kinematics_c pose;

void setup() {

  // Serial for debugging.
  Serial.begin(9600);
  delay(1000);

  // Initialises the routines that will automatically
  // count the rotations of the wheels.
  setupEncoder0();
  setupEncoder1();

  // Initialise the pose of the robot to x=0, y=0,
  // theta=0.  
  pose.initialise( 0, 0, 0 );

}


void loop() {

  // Instruct the kinematics to perform an update
  // of the robot position
  // It is recommended you schedule this to occur at
  // a fixed interval, such as 20ms.
  pose.update();

  // Print x,y,theta on one line separated by commas
  Serial.print( pose.x );
  Serial.print(",");
  Serial.print( pose.y );
  Serial.print(",");
  Serial.print( pose.theta );
  Serial.print("\n");

  // It would be better to use non-blocking code
  // which were discussed in labsheet 0.
  delay(20);
}
```

The kinematics is the mathematics to estimate the robot location based on the number of encoder counts that have occured.  This technique is known as **dead reckoning**, a form of **odometry**.  

**Odometry** is a more general term and means to use sensors to estimate a change of position over time.  This could include sensors such as gyroscopes, accelerometers, etc - which are not covered in these labsheets.  

**Dead-reckoning** is a more specific term, which informally means to attempt to "keep count" relative to a previously known or fixed position.  

It is important to understand the consequences of the following line of code, and therefore the interplay of local and global frames:

`pose.initialise( 0, 0, 0 );`

The above line of code is declaring that the robot starts at the global position of $x=0, y=0, \theta=0$, with respect to the origin.  Remember that for our simple 3Pi+ robot,  it receives it's initialisation from you - it has no other source of global information.  **Therefore, the global frame (global coordinate system) is defined by the position and direction that you place the robot before you power it on**:






<p align="center">
<img src="https://github.com/paulodowd/SEMTM0042_43/blob/main/Images/3Pi_placement_frames.png?raw=true"> <br>
</p>

In the above illustration, we can see that the **global frame** is denoted by grey arrows and $X_I, Y_I$.  The subscipt $I$ is simply convention to indicate the **global frame**.  We can think of the global frame as the external (outside the robot) point of reference against which the robot is going to be moving within.  The robot would report it's position as a coordinate on this global frame.  In our case, where the robot is powered-on becomes the origin of it's global frame.

The front of the robot and the **configuration** of it's parts is given by the **local frame**, shown as red and green arrows, and $X_R, Y_R$.  Again, subscript $R$ is convention to indicate the **local frame**.  By configuration, we can think of describing the placement of a wheel with respect to the origin of $X_R, Y_R$.  This can be confusing, but consider: if the robot were to rotate, we expect the wheel (and all it's other parts) to rotate with it - and so, their description is relative to $X_R, Y_R$.

For our robot and in these labsheets, we align the local X axis ($X_R$) with the forward direction of the robot.  If you look closely at the circuit board on the robot, you will see that this set of axis are labelled in the top-left.  Therefore, from the robot's own point of view, when it moves forwards, it will be moving along it's $X_R$ axis.

In image **a)**, we can see that phyiscally placing the robot to face upward on the coursework map would mean that the **global frame** $X_I$ is positive in the upward direction.  Because we are using the line of code `pose.initialise(0,0,0);`, the robot **local frame** $X_R, Y_R$ is begins aligned to $X_I, Y_I$.  The global frame would remain in this direction, and when the robot begins moving the local frame will move within it.  Because of this initial situation, if the robot was to move forwards, it would report an increasing $X_I$.

In image **b)**, the robot $X_R, Y_R$ is still aligned to $X_I, Y_I$ (`pose.initialise(0,0,0);`), but the robot was physically placed facing rightward.  Therefore, the **global frame** has $X_I$ positive to the right.  This is because we didn't tell the kinematics of any alignment change between the global and local frame (the third argument of `pose.initialise(0,0,0);').  Therefore, the global frame will remain in this direction, and the local frame will move within it as before.  If the robot was to move forwards, it would report an increasing $X_I$ - but from a human point of view, $X_I$ is now facing right across the coursework map.

In image **c)**, the robot $X_R, Y_R$ was specified as initialising as rotated +90° (`pose.initialise(0,0,π/2);`), therefore the robot starts "knowing" that it's **local frame** $X_R$ has been rotated to align with the **global frame** $Y_I$ (90° from $X_{I}$).  The global frame will remain in this direction, the local frame would move within it, as before.  If the robot was to move forwards, it would report an increasing $Y_I$, because it was initialised with $X_R$ along $Y_I$.

In the above example code, we have the lines:

```c
  pose.update();

  // Print x,y,theta on one line separated by commas
  Serial.print( pose.x );
  Serial.print(",");
  Serial.print( pose.y );
  Serial.print(",");
  Serial.print( pose.theta );
  Serial.print("\n");
```

`pose.update()` performs the kinematic update based on encoder counts.  The `Serial.print()` lines are reporting the **global** co-ordinates that the robot estimates itself to be at.  








<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  If you have found these concepts difficult to grasp, it is recommended that you try running the above example code and observing the changes in the reported $X,Y,\theta$.  You can also try changing the line `pose.initialise(0,0,0);` and observe the effect.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  You are free to decide how to initialise your robot and co-ordinate frame.  You can adopt an approach that makes more sense to you.  Which way should the global $X_{I}$ face, do you think?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left">  If you would like more information on the kinematic model, a good source of further reading is the book: **Siegwart, R., Nourbakhsh, I.R., Scaramuzza, D., Arkin, R.C. (2011) Introduction to Autonomous Mobile Robots 2nd Edition.  MIT Press. ISBN-10 0262015358.** (<a href="https://bris.on.worldcat.org/oclc/708414202">available in the University of Bristol Library</a>)

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left">  The kinematic model provided in `Kinematics.h` is a simpler form which assumes that the robot will either translate or rotate (exclusively).  The kinematics model movement as a translation, followed by a rotation.  Therefore, the model will represent curved motion as a straight-line approximation.  Some error is inherent to the model.  However, the simple kinematic model has proven sufficient to complete Assessment 1 to a high standard.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left">  You can inspect the official documentation for the dimensions of the Pololu 3Pi+ robot, <a href="https://www.pololu.com/file/0J1827/3pi-plus-32u4-oled-robot-dimensions.pdf">here</a>.

<hr><br><br><br><br>

## Exercise 1: Calibrate Translation

The kinematics provided to you has the measurements <a href="https://www.pololu.com/product/4975/resources">provided by the manufacturer as the model parameters</a>.  These parameters can be improved by conducting a manual calibration.  It is recommended you complete this calibration with your robot at least once.  You can update the parameters directly in the `Kinematics.h` file.

<p align="center">
<img src="https://github.com/paulodowd/SEMTM0042_43/blob/main/Images/calib_translation.png?raw=true"> <br>
</p>



<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** To calibrate translation, you should choose a known distance to move your robot across forwards.  Then, push your robot **gently** with the motor power **off**.  Report the `pose.x` value and apply the above logic.  Do your best to keep your robot moving in a straight line.  



1. Ensure that `pose.update();` is being called within `loop()`. It is highly recommended that you schedule this function call using `millis()` to operate at an interval such as 20ms.  This technique was originally covered in Labsheet 0.

2. Test that your program is updating values within the kinematics class by using Serial.print() on the variables `pose.x`, `pose.y`, `pose.theta`.  
  - Add these print statments into `loop()`.
  - Remember that you can increase the precision of `Serial.print()` by adding a whole number as a second argument, e.g. `Serial.print( pose.x, 4 );`



3. **Calibrating Translation**: The coursework map has a grid with 10mm spacing between cells drawn in light grey.  Use this to calibrate the **x contribution** (forward translation) of your robot.  
  - Printers can vary!  It would be smart to verify that the printed grid does in fact have a 10mm spacing.
  - Position your robot with the axles of both wheels aligned over the line marked 0mm.  
  - **Without motor power**, push the robot along in a straight line, stopping at the line marked 100mm.
  - Inspect the output for the **x coordinate** from the pose instance of `Kinematics_c`.  You can do this simply with `Serial.print( pose.x, 4 );`.
  - If your x coordinate is more than 100mm, you will need to reduce the parameter `wheel_radius` in the file `Kinematics.h`.
  - If your x coordinate is less than 100mm, you will need to increase the parameter `wheel_radius` in the file `Kinematics.h`.
  - Repeat this process improving the reported value of X.

<p align="center">
<img width="250px" src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/3Pi_grid_start.jpg"> &nbsp; <img width="250px" src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/3Pi_grid_end.jpg"><br>
<small>The robot positioned at the start of the grid, with the wheel axle positioned over the line 0mm (left).  The robot at the end of the grid, with the wheel axle positioned over the line marked 100mm (right).
</p>

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** It would make sense that calling `pose.update()` as quickly as possible would reduce the error of a straight-line approximation of a curve (the curve would be represented by smaller segments, a "higher resolution").  However, this does not seem to be the case in reality.  Why might this be?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** There is slightly smarter way to calibrate your translation.  We have both the ground truth value (the distance you are testing against), and the distance being reported.  You can take the difference between these two values, and then you can substitute the robot dimensions to calculate the adjustment to make to the wheel radius value.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** The <a href="https://www.pololu.com/product/4975/resources">manufacturer documentation</a> for the Pololu 3Pi+ is quite detailed and includes important measurements for the robot.  Why might the reported measurements be inaccurate? What physical factors may influence the effect of these parameters on the kinematic model?

<hr><br><br><br><br>


## Exercise 2: Calibrate Rotation

Similar to the previous exercise, we will use a **ground truth** value of **rotation**, and then compare the reported value of `pose.theta` by the kinematics.   The easiest value to use to begin with is 90° (π/2 in radians).  

<p align="center">
<img src="https://github.com/paulodowd/SEMTM0042_43/blob/main/Images/calib_rotation.png?raw=true"> <br>
</p>

The above illustration shows the effect of the **wheel separation**.  The correct rotation of 90° is represented with black lines.  We can imagine that the wheel separation brings the wheel either closer or further from the point of rotation at the centre of the robot.  Importantly, the **radius of the wheel** remains the same.  In the above illustration, the green, black and blue dashed lines are equivalent arc lengths, which we can imagine as the wheel travelling the same distance.  If the wheel travel were to travel for the same distance (arc length), we can see that an incorrect wheel separation would calculate through as a body rotation too small or too large.  





<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:**  To conduct this calibration exercise you will move the physical robot from 0° to 90°, and the robot will report it's estimate of the rotation.  



4. **Calibrating Rotation:** The coursework map has a radial figure of angles in radians.  Use this to calibrate the rotation (**theta**) of your robot kinematics.
  - Position the robot exactly within the centre of the radial figure.  You can do this by lining up the wheel axles with a horizontal line, and by lining up the gap between the bumper levers with the vertical line.  
  - **Without motor power** rotate the robot to a pre-determined angle, such as 90 degrees ($\frac{\pi}{2}$, or 1.57 radians).
  - Inspect the reported value of `theta`.  
  - Increase or decrease the parameter `wheel_sep` (wheel seperation) within `Kinematics.h` depending on whether the reported value is too high or too low.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** This labsheet is instructing you to first calibrate translation.  Do you think the order of calibrating translation then rotation matters? If so, why might that be?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** There is slightly smarter way to calibrate your calibration.  We have both the ground truth value (the rotation you are testing against), and the rotation being reported.  You can take the difference between these two values, and then you can substitute the robot dimensions to calculate the adjustment to make to the wheel separation value.

<hr><br><br><br><br>


# Closed-loop Movement Primitives

After calibrating your kinematics, we are now ready to develop some simple movement routines to achieve precise rotation, and precise travel.  We call these **primitives** because we expect these to be the most basic movements our robot can make, and that we might chain or link together into more complicated motions.



<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  In the next exercises, you will want to draw on your previous work writing **non-blocking functions**.  It will be important that your robot can be operating a movement whilst also regularly and consistently updating the kinematics.   If it cannot update the kinematics regularly, your robot won't know when to stop.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left">  Whilst you might be able to use lots of `while()` loops, `break;`, and `delay()`, it will become difficult to debug your code, difficult to achieve flexible waypoint navigation for Assessment 1, and difficult to re-use your code in Assessment 2.  Invest the time to develop best practice.

<hr><br><br><br><br>


## Turning to a Precise Angle

In previous labsheets, you have been guided towards writing a **non-blocking** function to perform forwards travel, turning, or both - with a duration specified in milliseconds (_ms_).  You may have found values of PWM and a time duration to create a turn of 90 degrees, for example.  

A clear issue with this approach is that it is **open-loop**, meaning that the robot does not know if it has achieved the desired turn, and the performance may vary between robots, or within the robot's operational lifetime.  This can lead to a lot of frustration, where we feel that the robot "should" turn 90 degrees - but of course there is no reason it "should".

With **open-loop** control, various factors may easily effect performance, such as:
- surface friction
- incline of surface
- the mass of the robot
- variable time execution of your code
- changes to the power supply (batteries)

By instead using the kinematic model, we can now instruct the robot to perform a turn to face a desired angle by looking at the most recent value of `pose.theta`.  This means duration is no longer required, the robot can manage the turn operation for as long as necessary, until the angle is achieved - **closed-loop control**.







### Approaches to Turning

When we conduct the turn operation, there are potentially two different solutions to the task.  It is also interesting to consider these two solutions relative to two paradigms in robotics: **deliberative** control and **reactive** control.

### Deliberative Approach
In a more **deliberative** approach, it is possible for the robot to calculate, before moving, how far to rotate each wheel to achieve a precise turn.  

For example, if the robot begins facing 90°, and we want the robot to face 180°, we know that the required rotation of the whole robot is +90° (the difference).  We then know that the wheels are positioned an equal distance away from the centre of the robot, and so each wheel will need to travel an arc in opposite directions to have the robot rotate on the spot.  

Our calculation would effectively be a quarter  of the circumference of a circle traced by the wheels (90 is a quarter of 360), where the position of the wheels relative to the centre of the robot is the radius of the circle being traced.

If we know the distance a wheel must travel along for a 90° turn, we could easily convert this to a required number of encoder counts for each wheel to rotate.  To ensure the robot rotates on the spot, it would be good to control wheel velocity, and therefore calculate how much time is required at a wheel velocity to acheive the required encoder count (or distance).  Utilising a model like this is **inverse kinematics** : how to configure the robot to achieve a desired end effector position.   

### Reactive Approach

In a more **reactive** approach, we would care less about the exact pre-calculation of the movement operation, and instead focus more on the outcome (or sensed state).  In a reactive approach we iteratively react (respond) to the current state of the system.  In this approach, we would rely on the **forward kinematic** model to update the displacement of the robot as the wheels are turning at each time step.  

For example, for the same +90° turn, we would simply start the robot rotating on the spot, and monitor the value of theta as it changes.


### Turn to an angle, or minimise the difference to zero?

With either of the above approaches, we would need to calculate the difference between the current orientation of the robot and the desired orientation. Within this operation it is important to consider what could happen if we set the objective of achieving an exact target angle.

Consider the following scenario:
- We have a simple turn operation where our robot always rotates to the left.
- Our robot started facing theta = 270°
- We instructed it to reach the target theta = 358°.
- Our robot rotated too quickly, or there was an unexpected delay in our program.
- The robot overshot (turned too far), and the kinematic model updated as theta = 2°.
- The robot now believes it needs to continue rotating in the same direction, because 2° is still less than 358°.
- This could repeat endlessly, and you'd watch your robot spinning on the spot.






<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> It is important to recognise that you will not necessarily have good control of when and how your robot will begin and end turning operations.  In Assessment 1, your robot will be navigating the coursework map for 4minutes.  Therefore, it would be difficult to guarantee that your robot would not get into the situation described above. A robust solution that can handle any pair of angles is required!



<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  Considering the above, it is much more useful to program a behaviour that has the objective of minimising the difference between two angles to zero.  If the difference is zero, the robot must be facing the target angle.  If we can compute the difference between two angles, we might be able to:
- mitigate problems of angles near to 0°/360°
- use the **sign (+/-) of the difference** to decide which way to rotate.
- the **magnitude of the difference** to decide what velocity to rotate at.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left">  The following section describes a particular challenge with **calculating the difference between two angles**.  Finding a good solution to this now, and rigorously testing your solution, will save you a lot of time debugging later.   Being able to rotate to any angle from a current angle of rotation will be **very** important.

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/3Pi_DiffAngles.png">
</p>


<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/theory.png" align="left"> In the above illustrations we see 3 different rotations of the 3Pi+ robot with respect to the global coordinate frame ($X_{I}$, $Y_{I}$).  The blue arrow represents the robots current rotation, $\theta_{I}$ (`pose.theta`).  The green arrow represents a desired target angle.  

$\theta_{L}$ represents the "large angle" of difference between $\theta_{I}$ ('pose.theta', blue arrow) and the desired angle of rotation (green arrow).

$\theta_{S}$ represents the "small angle" of difference between $\theta_{I}$ ('pose.theta', blue arrow) and the desired angle of rotation (green arrow).

In the first panel, the problem appears easy.  $\theta_{I}$ is zero, and the robot will rotate positively towards the green arrow.  If the green arrow was $45°$, then $45 > 0$ (i.e. target > $\theta_{I}$), and we could take this to mean rotate left.

In the second panel, we can imagine that $\theta_{I}$ is now 350°, and the green arrow (target angle) is 10°.  Now, 10° < 350° (i.e. target < $\theta_{I}$, **less than!**), and so the robot would rotate **right** - the long way around.  

Rotating the long way around is not necessarily an error or incorrect.  However, it is likely to cause you some motion control problems later, or to simply accrue more error integration in your kinematics.

Therefore, when we reconsider this problem, we can articulate it as **requiring the smallest angle between two angles**.  Let's call "the smallest angle between two angles" $\theta_{s}$.  To produce an effective turning behaviour, we therefore want to:
- work out the current value of $\theta_{s}$, for the current rotation $\theta_{I}$ and the desired angle.
- rotate in the correct direction to make the next calculation of $\theta_{s}$ smaller (minimisation problem)
- stop moving once $\theta_{s} = 0 $ (the target has been reached).

<hr><br><br><br><br>


## Exercise 3: Implementing & Testing an Angular Difference Function

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Your Pololu 3Pi+ is the **target platform** for all the code you will write. It can be quite surprising to find differences between library implementations and compilers.  It therefore makes good sense to try out some calculations on the platform itself.  Regarding the above information on calculating the difference between two angles:


- write a function for your 3Pi+ that will accept two floating type variables as arguments, to correspond to a target angle and a source angle.
- write your code to calculate the smallest difference between these two angles.
- write your function to return the result.
- use `Serial.print()` to view the result.
- write some code to provide different target angle and source angle inputs to the function.
- consider what ranges of values are sensible, possible and/or exceptional for your function, and test what result your function produces.
- you may want to write either some error handling, or methods to mitigate exceptional values.

<hr><br><br><br><br>


## Exercise 4: Implementing a Logical Turn Procedure

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> Let's work in **small steps**.  We will first of all complete some exercises to simply tell the robot to activate the motors to turn left or right based on the sign (+/-) of the calculated angular difference between the current robot rotation (`pose.theta`) and a desired angle. This will be a **logical controller** - a very simple controller.  We will then update this to try a simple **continuous controller**, where the motor demand will be adjusted in propoprtion to the angular difference.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> A **logical controller** is likely too simple to operate a precise turning behaviour.  Even if you can achieve very slow rotaton to reach a target angle, the overall performance for the foraging challenge may be too slow.  A **proportional controller** may be **good enough**.  You should use your own judgement to evaluate the performance of a proportional controller, and whether you want to proceed to Exercise 4 Implementating a PID Turn Procedure.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Using the **non-blocking** functions you have previously written for your robot movement:


- Add a global variable that represents `target_angle`.  Whilst we could change this value directly from anywhere in the code, this could lead to many debugging problems.  Instead, use the function `setTurn()` to set the value. This way, using the turn operation will become standardised and consistent.  
- You will want to call `setTurn()` once, when the turn operation is first required. After this, you will want to call `checkTurn()` to complete the turn in a non-blocking way.
- It is recommended you only update your kinematics at one location in your code, to keep the execution of your code as predictable and as consistent as possible.  A good location is therefore within `loop()`, because we will want to update the kinematics on every iteration of `loop()`.  Therefore, it should not be necessary to call `pose.update()` within your turning functions.
- Remember that the kinematics class (`Kinematics_c`) instance `pose` is defined as a global variable (you can find the line `Kinematics_c pose;`).  Therefore, your functions `setTurn()` and `checkTurn()` can access the instance `pose.` directly, you don't need a function argument for `pose`.  For example, `float a = pose.theta;` would store the current value of theta within pose.
- The issue of two angles crossing the 0/360 axis of a circle is a well understood problem.  The <a href="https://en.wikipedia.org/wiki/Atan2">atan2</a> function helps significantly in solving this problem.
- Implement calls to either your motor class to power the motors, or your PID speed controllers.  Set one motor to run forwards and one backwards.  This will generate a turn on the spot.  
- Remember to check the sign of the angular difference, and use this to turn left or right appropriately.
- Stopping precisely can be difficult.  Once you have a calculation for $\theta_{s}$, use an `if()` statement to check if this has reached a small enough value - a **threshold**.  For example:



```c
// You can define a value at the top of your program
// so that you don't have to search for it when you
// want to change it.
#define TURN_THRESHOLD 0.1

// Used to initialise a turn procedure, and "save"
// the desired target angle.
void setTurn( float target_angle ) {
  // ...
}

// A non-blocking function to update motor outputs
// with respect to achieving a desired angle of
// robot rotation.
bool checkTurn( ) {

  // Is the difference between current rotation and
  // target_angle now sufficiently small?
  // Function abs() returns the unsigned value.
  if( abs(angle_difference) < TURN_THRESHOLD ) {

    // Stop robot
    motors.setPWM( 0, 0 );

    // Signal turn complete.
    return true;

  }

  // Signal turn has not finished.
  return false;

} // End of checkTurn()

```

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left"> **Validate:** validate that your turning operation is working by trying various test cases.  It is likely that you cannot know in advance what rotation your robot will have when it must complete a turn towards a target angle.  Remember that you can be more efficient in your debugging by isolating parts of your code.  For example, you could call `setTurn()` within `setup()` so that it happens when your robot is powered-on.  And then simply call `checkTurn()` within `loop()` (alongside other necessary functions).  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/observation.png" align="left"> **Observe:** Try different PWM values for your motors.  If you have implemented PID speed controllers, try different values of fixed speed demands.  What happens to the robot behaviour when the turning velocity is (a) too fast (b) too slow?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Update your code to now map the **magnitude** or your calculated angular different to either a PWM value or a PID speed controller demand.  A code extract below provides a suggestion of how this might be achieved.  If you have completed the PID speed controller exercises in Labsheet 1, you should recognise this as a P-Controller.  After you have implemented the basic functionality, take the following steps:

- Start with a small **angular difference** value.  Either set this manually, or ask your robot to move between two rotation positions such as 90° to 100°, or 90° to 80°.
- Find a value for `turn_gain` that works reliably for this small rotation.  You will be effecitively scaling a small angular difference to a sensible PWM or speed value to generate motion.
- Investigate what happens as you increase the value of your angular difference.  Does your `turn_gain` value still give acceptable performance?
- What are the limits of your angular difference that produces reliable or acceptable turning behaviour?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> In the above exercise, you should observe that for a P-Controller, a given proportional gain $K_p$ will not necessarily work across very large input ranges.  There are many ways to attempt to solve or mitigate this issue.  In this case, the simplest approach is to apply **clamping**.  You can either (a) clamp the input (b) clamp the output.  Clamping means you limit the possible value.  For a P-Controller, clamping is relatively simple and safe.  However, for a full PID controller, clamping can cause significant issues and undesirable consequences.  

```c
// If using PWM

  // You will need to find a sensible value
  // for this variable
  float turn_gain = 10.0;

  float demand_turn_pwm = angular_difference * turn_gain;

  motors.setPWM( demand_turn_pwm, -demand_turn_pwm);


```

```c
// If using PID speed controllers

  // You will need to find a sensible value
  // for this variable
  float turn_gain = 10.0;

  float demand_turn_speed = angular_difference * turn_gain;

  float left_pwm = pid_left.update( demand_turn_speed, left_measured_speed);
  
  // Note: demand_turn_speed is inversed (-)
  float right_pwm = pid_left.update( -demand_turn_speed, left_measured_speed);
  
  motors.setPWM( left_pwm, right_pwm);
```

<hr><br><br><br><br>

## Exercise 5: Implementing a PID Turn Procedure

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> The following exercise is not strictly essential to achieve a good level of performance on your robot.  You may wish to skip this exercise, and come back later when you need to improve or refine the performance of your robot.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/theory.png" align="left">  A particular challenge around having your robot turn a precise angle is when the robot stops rotating and leaves a small amount of error.  When there is a small amount of rotation error, your motors will probably have moved into their **dead-band** region (not enough power to rotate), or the difference in rotation angle is not being mathematically manipulated to a useful speed demand.  It would therefore be useful if your robot could somehow "know" when it was stuck and respond.  This can be achieved with a PI-Controller (a controller that uses both the proportional term and the integral term).  If the robot has turned and stopped with a small amount of error, the i-term will continue to integrate this error over time.  At some point, the i-term will contribute enough feedback to cause the motors to operate outside of their dead-band, and the robot will rotate.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> The below block diagram illustrates a **nested PID controller design**.  In a nested controller, one portion of the control system is receiving it's input signal from the output of another controller.  In our system, we can have a **high level heading controller** which is attempting to correct the rotation of the robot.  To achieve this, it is generating **wheel speed demands** as it's output.  This output becomes the input to the PID speed controllers operating the motors.  

In the below diagram, notice that the summing block (⦻) for the Right PID Speed controller has two negative (-) symbols, whilst the Left PID Speed controller has a positive (+) and negative (-) symbol.  This summing block is communicating how the error signal is generated. In our system, the single heading controller output is used as an input to both the left and right PID speed controller sub-systems.  However, the input is inverted for one of the speed controller inputs, causing the robot to rotate on the spot with an equal speed on both motors but in opposite directions.  

<p align="center">
<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/nested_pid.png">
</p>

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left"> Using the functions you have developed in the previous exercises, you can use the below code extract to help you to:

- Create an instance of the PID class `PID_c` for the purpose of controlling the rotation of the robot.
- Initialise the PID class instance to have 0 integral gain and 0 derivative gain.  Given that you are expecting to use an angular difference as the measurement, decide a reasonable proportional gain value to start with.  What range of values of the output of the heading controller will be useful?
- Utilise your calculated angular difference as the **measurement** of your rotation procedure.
- **Use 0 as your demand** - we can think of this as asking the robot to minimise the measurement (angular difference) to reach the desired target.
- Start with initialising the proportional gain ($K_p$), and check that your robot is rotating in the right direction with respect to the error.
- Once you are satisfied with the proportional term, introduce a very small integral gain ($K_i$).  Whlist it may take a long time, you should be able to see your robot eventually attempt to correct a steady-state error in it's rotation.  Check that this integral term is providing feedback to rotate the robot in the correct direction.
- Incrementally increase and test your integral gain until you achieve good performance.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** In the above exercise, you're guided to use a PID controller to minimise the angular difference to 0.  The alternative approach is to use exact target angles.  What difficulties might arise if you use exact target angles?  It might help to think up a variety of target angles and how the error signal would be constructed.

```c
#include "PID.h"

PID_c left_pid;  // To control the left motor
PID_c right_pid; // To control the right motor

PID_c heading;   // To control turning operations


// A non-blocking function to rotate the robot
// until a desired kinematics theta angle is
// achieved.
void checkTurn( ) {

  // Have we finished the turn? If not:
  if( abs(angle_diff) > TURN_TOLERANCE ) {

    // Generate a speed demand for rotation
    // based on a demand and measured theta
    // angle from kinematics.
    // Note, the following approach will minimise
    // the angular difference between the current
    // theta in kinematics and the desired angle.
    float demand_turn_speed = heading.update( 0, angle_diff );

    // Generate the required PWM signal based
    // on a speed demand and measurement.
    // Note, the demand_turn_speed is given with
    // opposite sign (+/-) for each wheel, causing
    // the rbot
    float l_pwm = left_pid.update( demand_turn_speed, left_measurement );
    float r_pwm = right_pid.update( -demand_turn_speed, right_measurement );

    // Send the PWM output to the left and right motor
    motors.setPWM( l_pwm, r_pwm );

    // Signal that the turn is still in progress
    // by returning "false"
    return false;

  } else {

    // Turn finished, stop the robot.
    stopRobot();

    // Signal that the turn finished by
    // returning "true".
    return true;

  }

}




```

<hr><br><br><br><br>


# Move a Distance, or Move to target X,Y coordinates?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/theory.png" align="left">  
At first it may seem like a good or simple solution is perform two simple robot behaviours one after the other.  

First rotate the robot to face a target X,Y coordinate.  Second, command the robot to move the required (fixed) distance.  With this approach, we can imagine a very sequential code design that computes the rotation and distance, and then assumes these were correct (and discards them).   

<p align="center">
<img src="https://github.com/paulodowd/SEMTM0042_43/blob/main/Images/iterative_travel.png?raw=true">
</p>

Whilst simple, the main problem with this approach is that a small error in the initial rotation will become amplified by the distance travelled.  Therefore, it is **not safe to assume** that calculating the rotation and distance just once is sufficient.  

By attempting to first rotate, and then travel, you will also encounter the task of requiring your robot to reliably drive in a straight line.  This can be achieved with good PID control.  However, it may be simpler to instead update motor control iteratively with respect to a target x,y co-ordinate.

As illustrated above, it is advantageous to instead approach the problem of travelling with an **iterative** solution.  At each timestep, the robot should both rotate and travel.  This approach takes advantage of new information provided by updating the kinematics at each time step.

However, this approach means we are no longer travelling a pre-calculated fixed distance.  The distance could infact vary.  Therefore, we would now calculate the distance to the target at each time step, and we can consider the movement complete when this distance is 0.

To achieve the **iterative solution**, your code implementation will need to "remember" what the target x and y coordinates are.  This is simple to implement with global variables - just like you were guided to with the turning behaviour.  

Similar to all previous exercises, you are highly encouraged to write a **non-blocking** solution.  Therefore, you may want to adopt a similar template of two functions:
- `setTravel(x,y)`: used to set a target x,y coordinate, and to initialise any other important variables or functions.
- `checkTravel()`: used to update the wheel speeds of the robot, to stop the robot if it has achieved the target, and may return `true` or `false` to indicate the current state of travel (i.e., complete or incomplete).




<hr><br><br><br><br>

## Exercise 6: Moving Forwards to X,Y Coordinates.

 The next set of exercises will start relatively simple but soon require you to integrate your solution to turning a precise angle.  The approach you take may limit your ability to diagnose and solve problems.  The next section of theory is intended to help you to think about the concepts and to set a good goal before you proceed.


<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/hypothesis.png" align="left">**Hypothesise:** Consider a simple solution that rotates once, then travels a distance.  Even if your robot was extremely good at achieving a precise rotation to a desired angle, what would be a potential remaining cause of error for the resultant travel?



<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> Use the above theory to set yourself your high level goal: to achieve an iterative non-blocking function for travel to an x,y coordinate that also adjusts rotation.  **However**, start simple!  Don't try to solve everything at once.  The following exercises will help you to build your solution in smaller steps.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Similar to the last exercise on turning, implement a non-blocking function for the purpose of having your robot move forwards.  **Start simple:** Assume that your robot initialises with $x=0, y=0, \theta=0$, and set your target coordinate to be something like $x=300, y=0$ (therefore, in front of the robot).  You are encouraged to use the same general approach of a function like `setTravel()` and `checkTravel()`.  When `setTravel()` has been called once to set ("remember") some x,y coordinates, your `checkTravel()` function will need to:




- calculate the distance between the robots current position (`pose.x`, `pose.y`) and the target position.
- if the distance is not zero, set the motor speed or power to move the robot forwards.
- if the distance is zero, set the motor speed or power to be 0 (stop the robot).





<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left"> **Validate:** Try this simple function have the robot move towards an x,y coordinate and stop when the calculated distance is 0. What do you notice?  It is very likely you will need to validate the performance and respond to some issues, such as:

- Have the robot stop at a **near-zero** distance.   You can use a **global variable** to set a value of distance (called a **tolerance**) where the robot will consider itself at the target.  At the moment, what is the smallest reliable value for your travel tolerance?
- Try different speeds of movement.  How does this effect performance?
- Check what happens if your robot is initialised facing in the wrong direction, or if your target is behind the robot.  What would happen to the calculated distance at each time step? What would be an appropriate response in this condition?

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** Drawing on your experience of writing your turning behaviours, look to integrate a response to the rotation of the robot within `checkTravel()`.  **Start Simple:** do not try large erorrs in rotation to begin with - just see if adding this functionality improves performance generally to reach an x,y position.  You will need to:


- calculate the angle between the robot's current position (x,y), and the target position (x,y).  Let's represent this angle as $\theta_D$, where D means "destination".  We can imagine this as the angle of the line between the robot and the target, the desired line of travel.
- calculate the angular difference between `pose.theta` ($\theta_I$) and $\theta_D$.  This is the difference between the way the robot is currently facing and desired line of travel.  Let's represent this as $\theta_L$, for the angle to the line of desired travel.
- use $\theta_L$ to add/subtract a correction to your wheel speed or motor power (which ever you are using).  You will need to **find a co-efficient** to scale this $\theta_L$ to be in an appropriate range of correction for your normal speed or power value.  For example:

```c
bool checkTurn() {
  // Your code
  // ...

  // (for example) normal speed of travel
  // is +0.1 for forward motion
  float fwd = 0.1;

  // Scale our theta_d by 0.1 to provide
  // a correction in a useful range.
  // 0.1 is just an example, you would need
  // to find an appropriate value.
  float turn = theta_L * 0.1;

  // Set motor speeds, with correction
  // applied in opposite directions to cause
  // some rotation.
  // If theta_d is 0, this will just be
  // forward motion.
  float left  = fwd + turn;
  float right = fwd - turn;

  // more of your code
  // ...
}



<hr><br><br><br><br>

## Exercise 7: A travelToXY() function.

At this point in the labsheet, you should have developed **non-blocking** functions for the discrete behaviours of:
- rotating on the spot to a specific angle.
- moving forwards to a target X,Y location, whilst making small corrective adjustments to rotation.  




It is important to recognise that if you don't first rotate your robot on the spot to face a desired x,y coordinate, your robot will produce a very large movement, like so:

<p align="center">
<img width="400px" src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/3Pi_LongTurn.png" >
</p>



<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/info.png" align="left"> **Information:** There are a few other issues with the above movement.  To minimise error in kinematics, you will also want to be relatively efficient with the types of movement your robot produces.

It would now be good to consider how to wrap both of these into a high level **non-blocking** function to avoid the above illustration.  Therefore, you function should always perform the two steps:
1. perform a (large) rotation on the spot to target x,y coordinate
2. move towards the target x,y coordinate

If we can achieve this goal, then we will have a very flexible piece of code to implement **Waypoint Following** or **Path Following**, which are addressed in the final labsheet.



<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/exercise.png" align="left">  **Exercise:** At this stage in the labsheets, it is a good intellectual challenge for you to develop a **non-blocking function** to achieve this `travelToXY()` functionality.  Your design requirements for your `travelToXY()` function should be:

1. To iteratively call `travelToXY()` from `loop()`.  Therefore, `travelToXY()` should not use a `for()` loop, `while()` loop, or `delay()`.  
  - this means that when you call `travelToXY()`, it will then call a turn or movement function appropriately.
  - the turn or movement function should also be **non-blocking**.  This means that they will quickly update the motor control, and then **return** ("exit").
  - once the turn or movement function has returned, `travelToXY()` should also then return.
<br>

2. `travelToXY()` should use a global variable (or more than one), to track which stage of the travel process it is engaged in.  For example:
- the initial rotation to face a target x,y coordinate.
- forward motion to reach the target x,y coordinate.
- finished: the distance to the target x,y coordinate is near 0.
- failing to reach the target (an error condition).

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> **Portability:** It is very possible to gain the same functionality with **blocking** code.  However, you should consider the **portability** of your code.  This means whether it can be used in or on another application.  When you focus on Assessment 2, it would be a great benefit to you if you can easily re-use your code from Assesment 1.  

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> **Bugs/Debugging:** If you write blocking code, it will be necessary for you to also make many calls to functions such as `pose.update()` and `lineSensors.readSensors()` in many locations throughout your code.  This has some **serious consequences**.  It will become very difficult to understand the **execution timing** of your code - meaning that some parts will take longer than others, or other parts may be called more often than others.  With many repeated calls, it will become easy to forget where they all are, or if some need updating, or if some are setting variables.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/tick.png" align="left"> **Non-Blocking:** By taking a non-blocking approach, you can also set yourself the goal of only making 1 call to `pose.update()` in your entire program.  There are good reasons to do this.  First, it means that you will always know where and when that call happens.  Second, you are much more likely to be able to make this function call at a regular time interval.  Both of these examples will help you to have **high confidence** in how and why your robot behaves the way it does.

<img src="https://raw.githubusercontent.com/paulodowd/SEMTM0042_43/main/Images/validate.png" align="left"> **Validate:** At this stage, we now want to start testing our motion control code in more challenging situations.  Previously, the recommendation was to use a target of $x=300, y=0$, which would be straight ahead of your robot when it is powered-on. In these exercises, it is much more interesting and challenging to set your robot's initial position (remember, `pose.initialise(0,0,0);`) with non-zero $x,y$ values.  Then, ask your robot to travel to $x=0, y=0$ (the origin).  This will generate a unique turn requirement at the start of each movement.

<hr><br><br><br><br>
